# Bias Variance

![title](ml_bias_variance.png)

Мы не будем выписывать строгие формулы, но попробуем объяснить идею этих понятий.

Пусть у нас есть алгоритм обучения, который по данным может создать модель.

Ошибка этих моделей может быть разложена на три части:
* **Noise** – шум данных, не предсказуем, теоретический минимум ошибки
* **Bias** – смещение, на сколько хорошо работает средний алгоритм. Средний алгоритм это "возьмём случайные данные, обучим алгоритм, сделаем предсказания", **Bias** – это ошибка средних предсказаний.
* **Variance** – разброс, на сколько устойчиво работает алгоритм. Опять же "возьмём случайные данные, обучим алгоритм, сделаем предсказания", **Variance** – это разрос этих предсказаний.

# Бустинг и Бэггинг в терминах Bias и Variance

Как вы думаете на какую составляющую Бустинг и Бэггинг влияют, а на какую нет?

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

/Users/chukrello/python3_env/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
data = pd.read_csv('HR.csv')

In [3]:
data.head()

,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
0,0.53,2,157,3,0,1,0
1,0.86,5,262,6,0,0,0
2,0.88,7,272,4,0,1,0
3,0.87,5,223,5,0,1,0
4,0.52,2,159,3,0,1,0


In [4]:
X, y = data.drop('left', axis=1).values, data['left'].values

In [5]:
data['left'].unique()

array([1, 0])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
def sample_model(seed, model):
    random_gen = np.random.RandomState(seed)
    indices = random_gen.choice(len(y_train), size=len(y_train), replace=True)
    model.fit(X_train[indices, :], y_train[indices])
    return model

def estimate_bias_variance(model, iters_count=100):
    y_preds = []
    for seed in range(iters_count):
        model = sample_model(seed, model)
        y_preds.append(model.predict(X_test))
    y_preds = np.array(y_preds)
    
    print('Bias:', np.mean((y_test - y_preds.mean(axis=0)) ** 2))
    print('Variance:', y_preds.std(axis=0).mean())

**Линейная регрессия**

In [8]:
estimate_bias_variance(LinearRegression())

Bias: 0.22051855839573647
Variance: 0.011108079724631232


**Решающее дерево с max_depth=5**

In [9]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=5))

Bias: 0.16342786779138255
Variance: 0.04217484196117754


**Решающее дерево с max_depth=10**

In [10]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=10))

Bias: 0.16302756126335638
Variance: 0.11966445821005375


**Решающее дерево с max_depth=15**

In [11]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=15))

Bias: 0.1704542728945588
Variance: 0.2244411489655311


**Решающее дерево без ограничения глубины**

In [12]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=None))

Bias: 0.19616773554056666
Variance: 0.3264994411420663


**Случайный лес n_estimators=1**

In [13]:
estimate_bias_variance(RandomForestRegressor(n_estimators=1, random_state=42))

Bias: 0.18418909545910134
Variance: 0.3587969574283942


**Случайный лес n_estimators=10**

In [14]:
estimate_bias_variance(RandomForestRegressor(n_estimators=10, random_state=42))

Bias: 0.18329082397384378
Variance: 0.1723060552291574


**Случайный лес n_estimators=50**

In [15]:
estimate_bias_variance(RandomForestRegressor(n_estimators=50, random_state=42))

Bias: 0.1829131313193182
Variance: 0.14173294774748665


**XGBRegressor**

**Бустинг над деревьями max_depth=20**

In [16]:
estimate_bias_variance(XGBRegressor(n_estimators=1, max_depth=20))

[14:44:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[14:44:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

**Бустинг над деревьями max_depth=10**

In [17]:
estimate_bias_variance(XGBRegressor(n_estimators=1, max_depth=10))

[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[14:44:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

**Бустинг над деревьями max_depth=5**

In [18]:
estimate_bias_variance(XGBRegressor(n_estimators=1, max_depth=5))

[14:44:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

**Бустинг над деревьями n_estimators=10**

In [19]:
estimate_bias_variance(XGBRegressor(n_estimators=10, max_depth=5))

[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[14:44:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

**Бустинг над деревьями n_estimators=100**

In [20]:
estimate_bias_variance(XGBRegressor(n_estimators=100, max_depth=5))

[14:44:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[14:44:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

**CatBoostRegressor**

In [21]:
estimate_bias_variance(CatBoostRegressor(n_estimators=1, max_depth=6, verbose=False))

Bias: 0.22059669136759513
Variance: 0.004656517477391416


In [22]:
estimate_bias_variance(CatBoostRegressor(n_estimators=10, max_depth=6, verbose=False))

Bias: 0.20050235674797137
Variance: 0.00688162924316269


In [23]:
estimate_bias_variance(CatBoostRegressor(n_estimators=100, max_depth=6, verbose=False))

Bias: 0.16545346114705772
Variance: 0.020550011425276647


**LGBMRegressor**

In [24]:
estimate_bias_variance(LGBMRegressor(n_estimators=1, max_depth=5))

Bias: 0.21294945912817415
Variance: 0.006742831808410252


In [25]:
estimate_bias_variance(LGBMRegressor(n_estimators=10, max_depth=5))

Bias: 0.17190753737382342
Variance: 0.020829163742335035


In [26]:
estimate_bias_variance(LGBMRegressor(n_estimators=100, max_depth=5))

Bias: 0.16265061665412614
Variance: 0.05505710640951436
